# Airborne Time-Domain

This survey can be used to store airborne time-domain electromagnetic (ATEM) data defined by a fixed transmitter-receiver loop configuration. The survey is made up of two entities ([AirborneTEMTransmitters](api/geoh5py.objects.surveys.electromagnetics.rst#geoh5py.objects.surveys.electromagnetics.airborne_tem.AirborneTEMTransmitters) and [AirborneTEMReceivers](..api/geoh5py.objects.surveys.electromagnetics.rst#geoh5py.objects.surveys.electromagnetics.airborne_tem.AirborneTEMReceivers)) linked by their metadata.

The following example shows how to generate an an airborne TEM survey with associated data programmatically.

![mtSurvey](./images/mt_survey.png)

In [1]:
import numpy as np
from geoh5py.workspace import Workspace
from geoh5py.objects import AirborneTEMReceivers, AirborneTEMTransmitters

# Create a new project
workspace = Workspace("my_project.geoh5")

# Define the pole locations
n_stations = 9
n_lines = 2
x_loc, y_loc = np.meshgrid(np.linspace(0, 60, n_stations), np.linspace(-20, 20., n_lines))
vertices = np.c_[x_loc.ravel(), y_loc.ravel(), np.zeros_like(x_loc).ravel()]

# Assign a line ID to the poles (vertices)
parts = np.kron(np.arange(n_lines), np.ones(n_stations)).astype('int')

# Create the survey as a coincident loop system
aem_receivers = AirborneTEMReceivers.create(workspace, vertices=vertices, parts=parts)
aem_transmitters = AirborneTEMTransmitters.create(workspace, vertices=vertices, parts=parts)

We have so far created two seperate entities, one for transmitter locations and another for the receivers. In order to finalize the survey, the association must be made between the two entities:

In [2]:
aem_receivers.transmitters = aem_transmitters

or equivalently

In [3]:
aem_transmitters.receivers = aem_receivers

Only one of the two options above is needed. 

Once linked, the two entities will share changes applied to the metadata. For example, changing the `input_type` property on the transmitters yield:

In [4]:
aem_transmitters.input_type = "Tx and Rx"
print(aem_receivers.input_type)

Tx and Rx


## Metadata

Along with the survey object itself, the metadata contains all the necessary information to define the geophysical experiment.

In [5]:
aem_receivers.metadata

{'EM Dataset': {'Channels': [],
  'Input type': 'Tx and Rx',
  'Property groups': [],
  'Receivers': UUID('f3eeae9b-675b-434f-880e-76ebc6b424a3'),
  'Survey type': 'Airborne TEM',
  'Transmitters': UUID('a4a563d2-9462-4669-a1ec-c3867d8306bc'),
  'Unit': 'Milliseconds (ms)',
  'Yaw value': 0,
  'Pitch value': 0,
  'Roll value': 0,
  'Loop radius': 1}}

### Channels

List of time channels at which the data are provided.

In [6]:
aem_receivers.channels = np.logspace(-3, -2, 10) # Simple sweep from 1 to 10 ms

### Input type

Label defining how the survey was created.

- `Rx`: Survey defined from the `AirborneTEMReceivers` positions, with the`AirborneTEMTransmitters` added from offsets.
- `Tx`: Survey defined from the `AirborneTEMTransmitters` position, with the`AirborneTEMReceivers` added from offsets.
- `Tx and Rx`: Survey defined by both the `AirborneTEMTransmitters` and the`AirborneTEMReceivers` positions.

### Property groups

List of [PropertyGroup](../api/geoh5py.groups.rst#module-geoh5py.groups.property_group)s defining the various data components (e.g. `dBzdt`, `Bz`, ...). It is expected that each component contains data channels at all times and in the same order as defined in `Channels`.

The class method [add_component_data](../api/geoh5py.objects.surveys.electromagnetics.rst#geoh5py.objects.surveys.electromagnetics.base.BaseEMSurvey.add_component_data) can help users add data from nested dictionaries. Below is an example using four components:

In [7]:
# Create some simple data
data_fun = lambda f: (f) * np.sin(np.pi * (x_loc * y_loc).ravel() / 200.)

# Create a nested dictionary of time data.
data = {
    "dBdt" : {
        f"time[{tt}]": {"values": data_fun(time)} for tt, time in enumerate(aem_receivers.channels)
    }
}
    
aem_receivers.add_component_data(data)

Metadata are also updated to reflect the addition of component data.

In [8]:
aem_receivers.metadata

{'EM Dataset': {'Channels': [0.001,
   0.001291549665014884,
   0.0016681005372000592,
   0.0021544346900318843,
   0.0027825594022071257,
   0.003593813663804626,
   0.004641588833612777,
   0.005994842503189409,
   0.007742636826811269,
   0.01],
  'Input type': 'Tx and Rx',
  'Property groups': ['dBdt'],
  'Receivers': UUID('f3eeae9b-675b-434f-880e-76ebc6b424a3'),
  'Survey type': 'Airborne TEM',
  'Transmitters': UUID('a4a563d2-9462-4669-a1ec-c3867d8306bc'),
  'Unit': 'Milliseconds (ms)',
  'Yaw value': 0,
  'Pitch value': 0,
  'Roll value': 0,
  'Loop radius': 1}}

### Receivers

Generic label used for surveys to identify the receiver entity. References to itself in the case of `AirborneTEMReceivers`.

### Survey type

Label identifier for `Airborne TEM` survey type.

### Transmitters

Generic label used for surveys to identify the transmitter entity. References to itself in the case of `AirborneTEMTransmitters`.

### Unit

Units for time sampling of the data: `Seconds (s)`, `Milliseconds (ms)`, `Microseconds (us)` or `Nanoseconds (ns)`.

### Loop radius

Specifies the transmitter loop radius.

### Custom fields

`Metadata` are stored in `geoh5` as a `json` structure allowing for custom data fields to be added to the survey: such as flight information, date/time, offsets, etc. Associated values can be of type `string`, `float` and `int`.

In [9]:
aem_receivers.yaw = 15.
aem_receivers.metadata["EM Dataset"]['Yaw value']

15.0

Aternatively, a `uuid.UUID` value can be used if the information is to be provided at every survey position. `Geoscience ANALYST` will automatically create a link referencing to the data entity under the project tree.

For certain metadata, such as `yaw`, `pitch`, `roll`, the suffix `property` is used instead of `value`:

In [10]:
yaw_data = aem_receivers.add_data({
    "yaw angles": {"values": np.random.randn(aem_receivers.n_vertices)}
}) # Add random data for yaw angles
aem_receivers.yaw = yaw_data.uid  # Assign to the yaw property
aem_receivers.metadata["EM Dataset"]['Yaw property']  # Metadata is updated and re-labeled.

UUID('ae485382-6157-4f6f-8a05-e15839a08da0')

In [11]:
workspace.finalize()